In [1]:
import duckdb
import time

In [2]:
def parse_float(s: str):
    try:
        return float(s), True
    except Exception:
        return None, False

def values_close(a: str, b: str, abs_tol: float = 1e-6, rel_tol: float = 1e-9) -> bool:
    av, a_is_num = parse_float(a)
    bv, b_is_num = parse_float(b)
    if a_is_num and b_is_num:
        # Both values numeric: compare with tolerance
        return abs(av - bv) <= max(abs_tol, rel_tol * max(abs(av), abs(bv)))
    # Fallback to exact string match
    return a == b

def rows_equal(row, expected_row) -> bool:
    if len(row) != len(expected_row):
        return False
    for i in range(len(row)):
        if not values_close(row[i], expected_row[i]):
            return False
    return True

def is_match(aggregate, original) -> bool:
    if len(aggregate) != len(original):
        return False
    for i in range(len(aggregate)):
        if not rows_equal(original[i], aggregate[i]):
            print(f"Mismatch at row {i}")
            print(f"Original: {original[i]}")
            print(f"Aggregate: {aggregate[i]}")
            return False
    return True

In [3]:
con = duckdb.connect('tmp/baseline.duckdb')
con.execute("SET timezone = 'America/Los_Angeles';")

In [4]:
con.execute(
    """
    CREATE OR REPLACE TABLE events_bids_minutes AS
        SELECT
            minute,
            ANY_VALUE(hour) as hour,
            ANY_VALUE(day) as day,
            ANY_VALUE(week) as week,
            SUM(bid_price) AS sum_bid_price,
            SUM(CASE WHEN type = 'impression' THEN 1 ELSE 0 END) AS count_impressions,
        FROM EVENTS
        GROUP BY minute
        HAVING count_impressions > 0;
    """
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
tic = time.time()
out_aggregate = con.execute("SELECT minute, sum_bid_price FROM events_bids_minutes ORDER BY minute").fetchall()
toc = time.time()
print(f"Aggregate time: {toc - tic:.3f}s")

tic = time.time()
out_original = con.execute("SELECT minute, SUM(bid_price) FROM events WHERE type = 'impression' GROUP BY minute ORDER BY minute").fetchall()
toc = time.time()
print(f"Original time: {toc - tic:.3f}s")

print(f"Correctness: {is_match(out_aggregate, out_original)}")

Aggregate time: 0.400s
Original time: 1.763s
Correctness: True


In [6]:
tic = time.time()
out_aggregate = con.execute("SELECT hour, SUM(sum_bid_price) FROM events_bids_minutes GROUP BY hour ORDER BY hour").fetchall()
toc = time.time()
print(f"Aggregate time: {toc - tic:.3f}s")


tic = time.time()
out_original = con.execute("SELECT hour, SUM(bid_price) FROM events WHERE type = 'impression' GROUP BY hour ORDER BY hour").fetchall()
toc = time.time()
print(f"Original time: {toc - tic:.3f}s")

print(f"Correctness: {is_match(out_aggregate, out_original)}")

Aggregate time: 0.159s
Original time: 1.377s
Correctness: True


In [7]:
tic = time.time()
out_aggregate = con.execute("SELECT day, SUM(sum_bid_price) FROM events_bids_minutes GROUP BY day ORDER BY day").fetchall()
toc = time.time()
print(f"Aggregate time: {toc - tic:.3f}s")


tic = time.time()
out_original = con.execute("SELECT day, SUM(bid_price) FROM events WHERE type = 'impression' GROUP BY day ORDER BY day").fetchall()
toc = time.time()
print(f"Original time: {toc - tic:.3f}s")

print(f"Correctness: {is_match(out_aggregate, out_original)}")

Aggregate time: 0.004s
Original time: 1.097s
Correctness: True


In [8]:
tic = time.time()
out_aggregate = con.execute("SELECT SUM(count_impressions) FROM events_bids_minutes GROUP BY day ORDER BY day").fetchall()
toc = time.time()
print(f"Aggregate time: {toc - tic:.3f}s")


tic = time.time()
out_original = con.execute("SELECT COUNT(*) FROM events WHERE type = 'impression' GROUP BY day ORDER BY day").fetchall()
toc = time.time()
print(f"Original time: {toc - tic:.3f}s")

print(f"Correctness: {is_match(out_aggregate, out_original)}")

Aggregate time: 0.095s
Original time: 1.031s
Correctness: True


In [9]:
tic = time.time()
out_aggregate = con.execute("SELECT SUM(count_impressions) as s FROM events_bids_minutes GROUP BY day ORDER BY s").fetchall()
toc = time.time()
print(f"Aggregate time: {toc - tic:.3f}s")


tic = time.time()
out_original = con.execute("SELECT COUNT(*) as s FROM events WHERE type = 'impression' GROUP BY day ORDER BY s").fetchall()
toc = time.time()
print(f"Original time: {toc - tic:.3f}s")

print(f"Correctness: {is_match(out_aggregate, out_original)}")

Aggregate time: 0.095s
Original time: 1.052s
Correctness: True


In [10]:
tic = time.time()
out_aggregate = con.execute("SELECT SUM(sum_bid_price)/SUM(count_impressions) FROM events_bids_minutes GROUP BY day ORDER BY day").fetchall()
toc = time.time()
print(f"Aggregate time: {toc - tic:.3f}s")


tic = time.time()
out_original = con.execute("SELECT AVG(bid_price) FROM events WHERE type = 'impression' GROUP BY day ORDER BY day").fetchall()
toc = time.time()
print(f"Original time: {toc - tic:.3f}s")

print(f"Correctness: {is_match(out_aggregate, out_original)}")

Aggregate time: 0.094s
Original time: 1.123s
Correctness: True


In [11]:
tic = time.time()
out_aggregate = con.execute("SELECT SUM(count_impressions) FROM events_bids_minutes GROUP BY day ORDER BY SUM(count_impressions)").fetchall()
toc = time.time()
print(f"Aggregate time: {toc - tic:.3f}s")


tic = time.time()
out_original = con.execute("SELECT COUNT(*) FROM events WHERE type = 'impression' GROUP BY day ORDER BY COUNT(*)").fetchall()
toc = time.time()
print(f"Original time: {toc - tic:.3f}s")

print(f"Correctness: {is_match(out_aggregate, out_original)}")

Aggregate time: 0.107s
Original time: 1.038s
Correctness: True


In [12]:
tic = time.time()
out_aggregate = con.execute("SELECT day, SUM(sum_bid_price) FROM events_bids_minutes GROUP BY day ORDER BY day").fetchall()
toc = time.time()
print(f"Aggregate time: {toc - tic:.3f}s")


tic = time.time()
out_original = con.execute("SELECT day, SUM(bid_price) FROM events WHERE type = 'impression' GROUP BY day ORDER BY day").fetchall()
toc = time.time()
print(f"Original time: {toc - tic:.3f}s")

print(f"Correctness: {is_match(out_aggregate, out_original)}")

Aggregate time: 0.003s
Original time: 1.103s
Correctness: True


In [13]:
optimize_bid_price_or_impression_count_query({
    "select": ["day", {"SUM": "bid_price"}],
    "from": "events",
    "where": [ {"col": "type", "op": "eq", "val": "impression"} ],
    "group_by": ["day"],
})

NameError: name 'optimize_bid_price_or_impression_count_query' is not defined

In [14]:
# Asbemble JSON query to SQL
# Note -- your solution may or may
# not need to use something similar depending on how you
# do query scheduling

def optimize_bid_price_or_impression_count_query(q):
    """
    Constructs queries that only aggregate on bid_price or counts impressions
    for optimization.
    """
    select = q.get("select", [])
    where = q.get("where", [])
    group_by = q.get("group_by", [])
    order_by = q.get("order_by", [])
    temporals = ["minute", "hour", "day", "week"]
    
    # WHERE clause must filter for impressions and columns of temporal type
    # has_only_impression_filter = (
    #     len(where) == 1 and
    #     where[0].get("col") == "type" and
    #     where[0].get("op") == "eq" and
    #     where[0].get("val") == "impression"
    # )
    has_only_impression_and_temporal_filter = False
    optimized_where = []
    for cond in where:
        if cond.get("col") == "type" and cond.get("op") == "eq" and cond.get("val") == "impression":
            has_only_impression_and_temporal_filter = True
        elif cond.get("col") in temporals:
            optimized_where.append(cond)
        else:
            has_only_impression_and_temporal_filter = False
            break

    if not has_only_impression_and_temporal_filter:
        # print("WHERE clause must filter for impressions and temporal columns")
        return False

    # GROUP BY must either be a temporal column or absent
    has_temporal_or_absent_group_by = (
        len(group_by) == 0 or
        (len(group_by) == 1 and group_by[0] in temporals)
    )
    if not has_temporal_or_absent_group_by:
        # print("GROUP BY must either be a temporal column or absent")
        return False

    # ORDER BY must be on a temporal column or absent
    # TODO: Extend to allow for SELECT aggregations
    has_temporal_or_select_aggregate_or_absent_order_by = (
        len(order_by) == 0 or
        (len(order_by) == 1 and order_by[0].get("col") in temporals)
    )
    if not has_temporal_or_select_aggregate_or_absent_order_by:
        # print("ORDER BY must be on a temporal column or absent")
        return False
    
    # Check aggregations in SELECT
    # Only aggregations (AVG or SUM) on bid_price or (AVG or SUM) on total_price or COUNT(*) or temporals
    optimized_select = []
    for item in select:
        if isinstance(item, dict):
            for func, col in item.items():
                # Allow COUNT(*) if filtering for impressions
                if col == "*" and func.upper() == "COUNT":
                    optimized_select.append("COUNT(count_impressions)")

                # Allow any aggregation on bid_price
                elif col == "bid_price" and func.upper() == "SUM":
                    optimized_select.append("SUM(sum_bid_price)")
                elif col == "bid_price" and func.upper() == "AVG":
                    optimized_select.append("SUM(sum_bid_price) / SUM(count_impressions)")

                # Reject other aggregations
                else:
                    # print("SELECT must be a single aggregation on bid_price or COUNT(*) or a temporal column")
                    return False
        elif isinstance(item, str):
            if item in temporals:
                optimized_select.append(item)
            else:
                # print("SELECT must be a single aggregation on bid_price or COUNT(*) or a temporal column")
                return False

    # If we get here, the query can be executed with our optimized path
    select_sql = _select_to_sql(optimized_select)
    specialized_tbl = "events_bids_minutes"
    where_sql = _where_to_sql(optimized_where)
    group_by_sql = _group_by_to_sql(group_by)
    order_by_sql = _order_by_to_sql(order_by)
    sql = f"SELECT {select_sql} FROM {specialized_tbl} {where_sql} {group_by_sql} {order_by_sql}"

    return sql.strip()

def assemble_sql(q, dark_launch=False):
    # check if query is optimized
    if dark_launch:
        optimized_sql = optimize_bid_price_or_impression_count_query(q)
        if optimized_sql:
            return optimized_sql.strip()

    select_sql = _select_to_sql(q.get("select", []))
    from_tbl = q["from"]
    where_sql = _where_to_sql(q.get("where"))
    group_by_sql = _group_by_to_sql(q.get("group_by"))
    order_by_sql = _order_by_to_sql(q.get("order_by"))
    sql = f"SELECT {select_sql} FROM {from_tbl} {where_sql} {group_by_sql} {order_by_sql}"
    if q.get("limit"):
        sql += f" LIMIT {q['limit']}"
    return sql.strip()

def _val_to_sql(col, op, val):
    def quote(val):
        # return f"'{val}'::event_type" if col == "type" else f"'{val}'"
        return f"'{val}'"
    match op:
        case "eq" | "neq":
            return quote(val)
        case "lt" | "lte" | "gt" | "gte":
            if col == "type":
                # This is a deviation from the baseline. If you gave the baseline
                # one of these ops it would put val directly into the SQL. But this
                # was a bug anyway for col == "type" since type *was* a VARCHAR and
                # its vals needed to be quoted.
                return quote(val)
            return val
        case "between":
            low, high = val
            return quote(low) + " AND " + quote(high)
        case "in":
            comma_separated = ", ".join(quote(v) for v in val)
            return f"({comma_separated})"

def _where_to_sql(where):
    if not where:
        return ""
    parts = []
    for cond in where:
        col, op, val = cond["col"], cond["op"], cond["val"]
        val_sql = _val_to_sql(col, op, val)
        if op == "eq":
            parts.append(f"{col} = {val_sql}")
        if op == "neq":
            parts.append(f"{col} != {val_sql}")
        elif op in ("lt", "lte", "gt", "gte"):
            sym = {"lt": "<", "lte": "<=", "gt": ">", "gte": ">="}[op]
            parts.append(f"{col} {sym} {val_sql}")
        elif op == "between":
            parts.append(f"{col} BETWEEN {val_sql}")
        elif op == "in":
            parts.append(f"{col} IN {val_sql}")
    return "WHERE " + " AND ".join(parts)


def _select_to_sql(select):
    parts = []
    for item in select:
        if isinstance(item, str):
            if item == "minute":
                parts.append("strftime(minute, '%Y-%m-%d %H:%M') AS minute")
            else:
                parts.append(item)
        elif isinstance(item, dict):
            for func, col in item.items():
                parts.append(f"{func.upper()}({col})")
    return ", ".join(parts)


def _group_by_to_sql(group_by):
    if not group_by: return ""
    return "GROUP BY " + ", ".join(group_by)


def _order_by_to_sql(order_by):
    if not order_by: return ""
    parts = [f"{o['col']} {o.get('dir', 'asc').upper()}" for o in order_by]
    return "ORDER BY " + ", ".join(parts)

In [17]:
for query in queries:
    optimized_sql = assemble_sql(query, dark_launch=True)
    original_sql = assemble_sql(query, dark_launch=False)

    print(optimized_sql)
    print(original_sql)
    
    if optimized_sql:
        optimized_out = sorted(con.execute(optimized_sql).fetchall())
        original_out = sorted(con.execute(original_sql).fetchall())

        if is_match(optimized_out, original_out):
            print("Query passed")
        else:
            print("Query failed")

SELECT day, SUM(sum_bid_price) FROM events_bids_minutes  GROUP BY day
SELECT day, SUM(bid_price) FROM events WHERE type = 'impression' GROUP BY day
Query passed
SELECT publisher_id, SUM(bid_price) FROM events WHERE type = 'impression' AND country = 'JP' AND day BETWEEN '2024-10-20' AND '2024-10-23' GROUP BY publisher_id
SELECT publisher_id, SUM(bid_price) FROM events WHERE type = 'impression' AND country = 'JP' AND day BETWEEN '2024-10-20' AND '2024-10-23' GROUP BY publisher_id
Query passed
SELECT country, AVG(total_price) FROM events WHERE type = 'purchase' GROUP BY country ORDER BY AVG(total_price) DESC
SELECT country, AVG(total_price) FROM events WHERE type = 'purchase' GROUP BY country ORDER BY AVG(total_price) DESC
Query passed
SELECT advertiser_id, type, COUNT(*) FROM events  GROUP BY advertiser_id, type ORDER BY COUNT(*) DESC
SELECT advertiser_id, type, COUNT(*) FROM events  GROUP BY advertiser_id, type ORDER BY COUNT(*) DESC
Query passed
SELECT strftime(minute, '%Y-%m-%d %H:%M'

BinderException: Binder Error: Could not choose a best candidate function for the function call "strftime(VARCHAR, STRING_LITERAL)". In order to select one, please add explicit type casts.
	Candidate functions:
	strftime(VARCHAR, TIMESTAMP) -> VARCHAR
	strftime(VARCHAR, TIMESTAMP_NS) -> VARCHAR
	strftime(VARCHAR, DATE) -> VARCHAR


LINE 1: SELECT strftime(minute, '%Y-%m-%d %H:%M') AS minute, SUM(sum_bid_p...
               ^

In [18]:
for i, query in enumerate(extended_queries):
    optimized_sql = optimize_bid_price_or_impression_count_query(query)
    if optimized_sql:
        print(query)
        print(i, optimized_sql)

{'select': ['day', {'SUM': 'bid_price'}], 'from': 'events', 'where': [{'col': 'type', 'op': 'eq', 'val': 'impression'}], 'group_by': ['day']}
0 SELECT day, SUM(sum_bid_price) FROM events_bids_minutes  GROUP BY day
{'select': ['minute', {'SUM': 'bid_price'}], 'from': 'events', 'where': [{'col': 'type', 'op': 'eq', 'val': 'impression'}], 'group_by': ['minute']}
4 SELECT strftime(minute, '%Y-%m-%d %H:%M') AS minute, SUM(sum_bid_price) FROM events_bids_minutes  GROUP BY minute
{'select': ['minute', {'SUM': 'bid_price'}], 'from': 'events', 'where': [{'col': 'type', 'op': 'eq', 'val': 'impression'}], 'group_by': ['minute'], 'order_by': [{'col': 'minute', 'dir': 'asc'}]}
27 SELECT strftime(minute, '%Y-%m-%d %H:%M') AS minute, SUM(sum_bid_price) FROM events_bids_minutes  GROUP BY minute ORDER BY minute ASC
{'select': ['minute', {'SUM': 'bid_price'}], 'from': 'events', 'where': [{'col': 'type', 'op': 'eq', 'val': 'impression'}, {'col': 'day', 'op': 'eq', 'val': '2024-01-01'}], 'group_by': ['min

In [16]:
queries = [
    {
        "select": ["day", {"SUM": "bid_price"}],
        "from": "events",
        "where": [ {"col": "type", "op": "eq", "val": "impression"} ],
        "group_by": ["day"],
    },
    {
        "select": ["publisher_id", {"SUM": "bid_price"}],
        "from": "events",
        "where": [
            {"col": "type", "op": "eq", "val": "impression"},
            {"col": "country", "op": "eq", "val": "JP"},
            {"col": "day", "op": "between", "val": ["2024-10-20", "2024-10-23"]}
        ],
        "group_by": ["publisher_id"],
    },
    {
        "select": ["country", {"AVG": "total_price"}],
        "from": "events",
        "where": [{"col": "type", "op": "eq", "val": "purchase"}],
        "group_by": ["country"],
        "order_by": [{"col": "AVG(total_price)", "dir": "desc"}]
    },
    {
        "select": ["advertiser_id", "type", {"COUNT": "*"}],
        "from": "events",
        "group_by": ["advertiser_id", "type"],
        "order_by": [{"col": "COUNT(*)", "dir": "desc"}]
    },
    {
        "select": ["minute", {"SUM": "bid_price"}],
        "from": "events",
        "where": [
            {"col": "type", "op": "eq", "val": "impression"},
            {"col": "day", "op": "eq", "val": "2024-06-01"}
        ],
        "group_by": ["minute"],
        "order_by": [{"col": "minute", "dir": "asc"}]
    }
]

extended_queries = [
  {
    "select": ["day", {"SUM": "bid_price"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "impression"}],
    "group_by": ["day"]
  },
  {
    "select": ["publisher_id", {"SUM": "bid_price"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "impression"}],
    "group_by": ["publisher_id"]
  },
  {
    "select": ["country", {"AVG": "total_price"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "purchase"}],
    "group_by": ["country"]
  },
  {
    "select": ["advertiser_id", "type", {"COUNT": "*"}],
    "from": "events",
    "group_by": ["advertiser_id", "type"]
  },
  {
    "select": ["minute", {"SUM": "bid_price"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "impression"}],
    "group_by": ["minute"]
  },
  {
    "select": ["type"],
    "from": "events",
    "where": [{"col": "country", "op": "eq", "val": "US"}]
  },
  {
    "select": ["advertiser_id"],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "serve"}]
  },
  {
    "select": ["publisher_id"],
    "from": "events",
    "where": [{"col": "bid_price", "op": "gt", "val": "0"}]
  },
  {
    "select": ["country"],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "purchase"}]
  },
  {
    "select": ["user_id"],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "impression"}]
  },
  {
    "select": ["day", {"SUM": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "impression"},
      {"col": "country", "op": "eq", "val": "JP"}
    ],
    "group_by": ["day"]
  },
  {
    "select": ["publisher_id", {"COUNT": "*"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "serve"},
      {"col": "country", "op": "eq", "val": "US"}
    ],
    "group_by": ["publisher_id"]
  },
  {
    "select": ["advertiser_id", {"AVG": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "impression"},
      {"col": "country", "op": "eq", "val": "KR"}
    ],
    "group_by": ["advertiser_id"]
  },
  {
    "select": ["country", {"SUM": "total_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "purchase"},
      {"col": "advertiser_id", "op": "eq", "val": "100"}
    ],
    "group_by": ["country"]
  },
  {
    "select": ["minute", {"COUNT": "*"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "serve"},
      {"col": "publisher_id", "op": "eq", "val": "500"}
    ],
    "group_by": ["minute"]
  },
  {
    "select": ["day", {"SUM": "bid_price"}],
    "from": "events",
    "where": [{"col": "day", "op": "between", "val": ["2024-01-01", "2024-01-31"]}],
    "group_by": ["day"]
  },
  {
    "select": ["publisher_id", {"COUNT": "*"}],
    "from": "events",
    "where": [{"col": "day", "op": "between", "val": ["2024-06-01", "2024-06-30"]}],
    "group_by": ["publisher_id"]
  },
  {
    "select": ["country", {"AVG": "total_price"}],
    "from": "events",
    "where": [{"col": "day", "op": "between", "val": ["2024-10-01", "2024-10-31"]}],
    "group_by": ["country"]
  },
  {
    "select": ["advertiser_id", {"SUM": "bid_price"}],
    "from": "events",
    "where": [{"col": "day", "op": "between", "val": ["2024-03-15", "2024-03-20"]}],
    "group_by": ["advertiser_id"]
  },
  {
    "select": ["type", {"COUNT": "*"}],
    "from": "events",
    "where": [{"col": "day", "op": "between", "val": ["2024-12-01", "2024-12-31"]}],
    "group_by": ["type"]
  },
  {
    "select": ["day", {"SUM": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "impression"},
      {"col": "country", "op": "eq", "val": "JP"},
      {"col": "day", "op": "between", "val": ["2024-10-20", "2024-10-23"]}
    ],
    "group_by": ["day"]
  },
  {
    "select": ["publisher_id", {"COUNT": "*"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "serve"},
      {"col": "country", "op": "eq", "val": "US"},
      {"col": "day", "op": "between", "val": ["2024-06-01", "2024-06-30"]}
    ],
    "group_by": ["publisher_id"]
  },
  {
    "select": ["advertiser_id", {"AVG": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "impression"},
      {"col": "country", "op": "eq", "val": "KR"},
      {"col": "day", "op": "between", "val": ["2024-03-01", "2024-03-31"]}
    ],
    "group_by": ["advertiser_id"]
  },
  {
    "select": ["country", {"SUM": "total_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "purchase"},
      {"col": "advertiser_id", "op": "eq", "val": "200"},
      {"col": "day", "op": "between", "val": ["2024-07-01", "2024-07-31"]}
    ],
    "group_by": ["country"]
  },
  {
    "select": ["minute", {"COUNT": "*"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "serve"},
      {"col": "publisher_id", "op": "eq", "val": "300"},
      {"col": "day", "op": "between", "val": ["2024-09-01", "2024-09-30"]}
    ],
    "group_by": ["minute"]
  },
  {
    "select": ["country", {"AVG": "total_price"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "purchase"}],
    "group_by": ["country"],
    "order_by": [{"col": "AVG(total_price)", "dir": "desc"}]
  },
  {
    "select": ["advertiser_id", "type", {"COUNT": "*"}],
    "from": "events",
    "group_by": ["advertiser_id", "type"],
    "order_by": [{"col": "COUNT(*)", "dir": "desc"}]
  },
  {
    "select": ["minute", {"SUM": "bid_price"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "impression"}],
    "group_by": ["minute"],
    "order_by": [{"col": "minute", "dir": "asc"}]
  },
  {
    "select": ["publisher_id", {"SUM": "bid_price"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "impression"}],
    "group_by": ["publisher_id"],
    "order_by": [{"col": "SUM(bid_price)", "dir": "desc"}]
  },
  {
    "select": ["country", {"COUNT": "*"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "serve"}],
    "group_by": ["country"],
    "order_by": [{"col": "COUNT(*)", "dir": "asc"}]
  },
  {
    "select": ["advertiser_id", "country", {"COUNT": "*"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "impression"}],
    "group_by": ["advertiser_id", "country"]
  },
  {
    "select": ["publisher_id", "type", {"SUM": "bid_price"}],
    "from": "events",
    "where": [{"col": "country", "op": "eq", "val": "US"}],
    "group_by": ["publisher_id", "type"]
  },
  {
    "select": ["day", "country", {"AVG": "total_price"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "purchase"}],
    "group_by": ["day", "country"]
  },
  {
    "select": ["advertiser_id", "publisher_id", {"COUNT": "*"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "serve"}],
    "group_by": ["advertiser_id", "publisher_id"]
  },
  {
    "select": ["type", "country", {"SUM": "bid_price"}],
    "from": "events",
    "where": [{"col": "day", "op": "between", "val": ["2024-01-01", "2024-01-31"]}],
    "group_by": ["type", "country"]
  },
  {
    "select": ["type", {"COUNT": "*"}],
    "from": "events",
    "where": [{"col": "country", "op": "eq", "val": "JP"}],
    "group_by": ["type"]
  },
  {
    "select": ["country", {"COUNT": "*"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "impression"}],
    "group_by": ["country"]
  },
  {
    "select": ["advertiser_id", {"COUNT": "*"}],
    "from": "events",
    "where": [{"col": "day", "op": "between", "val": ["2024-06-01", "2024-06-30"]}],
    "group_by": ["advertiser_id"]
  },
  {
    "select": ["publisher_id", {"COUNT": "*"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "serve"},
      {"col": "country", "op": "eq", "val": "US"}
    ],
    "group_by": ["publisher_id"]
  },
  {
    "select": ["day", {"COUNT": "*"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "purchase"}],
    "group_by": ["day"]
  },
  {
    "select": ["country", {"AVG": "bid_price"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "impression"}],
    "group_by": ["country"]
  },
  {
    "select": ["advertiser_id", {"AVG": "total_price"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "purchase"}],
    "group_by": ["advertiser_id"]
  },
  {
    "select": ["publisher_id", {"AVG": "bid_price"}],
    "from": "events",
    "where": [{"col": "country", "op": "eq", "val": "KR"}],
    "group_by": ["publisher_id"]
  },
  {
    "select": ["day", {"AVG": "total_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "purchase"},
      {"col": "country", "op": "eq", "val": "US"}
    ],
    "group_by": ["day"]
  },
  {
    "select": ["type", {"AVG": "bid_price"}],
    "from": "events",
    "where": [{"col": "day", "op": "between", "val": ["2024-03-01", "2024-03-31"]}],
    "group_by": ["type"]
  },
  {
    "select": ["country", {"SUM": "bid_price"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "impression"}],
    "group_by": ["country"]
  },
  {
    "select": ["advertiser_id", {"SUM": "total_price"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "purchase"}],
    "group_by": ["advertiser_id"]
  },
  {
    "select": ["publisher_id", {"SUM": "bid_price"}],
    "from": "events",
    "where": [{"col": "country", "op": "eq", "val": "CA"}],
    "group_by": ["publisher_id"]
  },
  {
    "select": ["day", {"SUM": "total_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "purchase"},
      {"col": "country", "op": "eq", "val": "JP"}
    ],
    "group_by": ["day"]
  },
  {
    "select": ["type", {"SUM": "bid_price"}],
    "from": "events",
    "where": [{"col": "day", "op": "between", "val": ["2024-07-01", "2024-07-31"]}],
    "group_by": ["type"]
  },
  {
    "select": ["day", {"SUM": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "impression"},
      {"col": "advertiser_id", "op": "eq", "val": "1"}
    ],
    "group_by": ["day"]
  },
  {
    "select": ["publisher_id", {"COUNT": "*"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "serve"},
      {"col": "country", "op": "eq", "val": "IN"}
    ],
    "group_by": ["publisher_id"]
  },
  {
    "select": ["country", {"AVG": "total_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "purchase"},
      {"col": "publisher_id", "op": "eq", "val": "999"}
    ],
    "group_by": ["country"]
  },
  {
    "select": ["advertiser_id", {"SUM": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "impression"},
      {"col": "country", "op": "eq", "val": "GB"}
    ],
    "group_by": ["advertiser_id"]
  },
  {
    "select": ["minute", {"COUNT": "*"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "serve"},
      {"col": "day", "op": "eq", "val": "2024-06-01"}
    ],
    "group_by": ["minute"]
  },
  {
    "select": ["advertiser_id", "country", "type", {"COUNT": "*"}],
    "from": "events",
    "where": [{"col": "day", "op": "between", "val": ["2024-01-01", "2024-12-31"]}],
    "group_by": ["advertiser_id", "country", "type"]
  },
  {
    "select": ["publisher_id", "country", {"SUM": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "impression"},
      {"col": "day", "op": "between", "val": ["2024-06-01", "2024-08-31"]}
    ],
    "group_by": ["publisher_id", "country"]
  },
  {
    "select": ["day", "type", {"AVG": "total_price"}],
    "from": "events",
    "where": [
      {"col": "country", "op": "eq", "val": "US"},
      {"col": "advertiser_id", "op": "between", "val": ["100", "200"]}
    ],
    "group_by": ["day", "type"]
  },
  {
    "select": ["country", "advertiser_id", {"COUNT": "*"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "serve"},
      {"col": "day", "op": "between", "val": ["2024-03-01", "2024-05-31"]}
    ],
    "group_by": ["country", "advertiser_id"]
  },
  {
    "select": ["type", "publisher_id", {"SUM": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "country", "op": "eq", "val": "JP"},
      {"col": "day", "op": "between", "val": ["2024-09-01", "2024-11-30"]}
    ],
    "group_by": ["type", "publisher_id"]
  },
  {
    "select": ["country", {"SUM": "bid_price"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "impression"}],
    "group_by": ["country"],
    "order_by": [{"col": "SUM(bid_price)", "dir": "asc"}]
  },
  {
    "select": ["advertiser_id", {"AVG": "total_price"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "purchase"}],
    "group_by": ["advertiser_id"],
    "order_by": [{"col": "AVG(total_price)", "dir": "desc"}]
  },
  {
    "select": ["publisher_id", {"COUNT": "*"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "serve"}],
    "group_by": ["publisher_id"],
    "order_by": [{"col": "COUNT(*)", "dir": "asc"}]
  },
  {
    "select": ["day", {"SUM": "total_price"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "purchase"}],
    "group_by": ["day"],
    "order_by": [{"col": "day", "dir": "desc"}]
  },
  {
    "select": ["type", {"AVG": "bid_price"}],
    "from": "events",
    "where": [{"col": "country", "op": "eq", "val": "US"}],
    "group_by": ["type"],
    "order_by": [{"col": "AVG(bid_price)", "dir": "asc"}]
  },
  {
    "select": ["minute", {"SUM": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "impression"},
      {"col": "day", "op": "eq", "val": "2024-01-01"}
    ],
    "group_by": ["minute"],
    "order_by": [{"col": "minute", "dir": "asc"}]
  },
  {
    "select": ["hour", {"COUNT": "*"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "serve"},
      {"col": "day", "op": "eq", "val": "2024-06-15"}
    ],
    "group_by": ["hour"],
    "order_by": [{"col": "hour", "dir": "asc"}]
  },
  {
    "select": ["minute", {"AVG": "total_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "purchase"},
      {"col": "day", "op": "eq", "val": "2024-12-25"}
    ],
    "group_by": ["minute"],
    "order_by": [{"col": "minute", "dir": "asc"}]
  },
  {
    "select": ["hour", {"SUM": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "impression"},
      {"col": "day", "op": "eq", "val": "2024-07-04"}
    ],
    "group_by": ["hour"],
    "order_by": [{"col": "hour", "dir": "asc"}]
  },
  {
    "select": ["minute", {"COUNT": "*"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "serve"},
      {"col": "day", "op": "eq", "val": "2024-03-17"}
    ],
    "group_by": ["minute"],
    "order_by": [{"col": "minute", "dir": "asc"}]
  },
  {
    "select": ["advertiser_id", {"SUM": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "country", "op": "eq", "val": "US"},
      {"col": "type", "op": "eq", "val": "impression"}
    ],
    "group_by": ["advertiser_id"],
    "order_by": [{"col": "SUM(bid_price)", "dir": "desc"}]
  },
  {
    "select": ["publisher_id", {"COUNT": "*"}],
    "from": "events",
    "where": [
      {"col": "country", "op": "eq", "val": "JP"},
      {"col": "type", "op": "eq", "val": "serve"}
    ],
    "group_by": ["publisher_id"],
    "order_by": [{"col": "COUNT(*)", "dir": "desc"}]
  },
  {
    "select": ["day", {"AVG": "total_price"}],
    "from": "events",
    "where": [
      {"col": "country", "op": "eq", "val": "KR"},
      {"col": "type", "op": "eq", "val": "purchase"}
    ],
    "group_by": ["day"],
    "order_by": [{"col": "day", "dir": "asc"}]
  },
  {
    "select": ["type", {"SUM": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "country", "op": "eq", "val": "CA"},
      {"col": "day", "op": "between", "val": ["2024-01-01", "2024-03-31"]}
    ],
    "group_by": ["type"],
    "order_by": [{"col": "SUM(bid_price)", "dir": "desc"}]
  },
  {
    "select": ["advertiser_id", {"AVG": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "country", "op": "eq", "val": "IN"},
      {"col": "type", "op": "eq", "val": "impression"}
    ],
    "group_by": ["advertiser_id"],
    "order_by": [{"col": "AVG(bid_price)", "dir": "asc"}]
  },
  {
    "select": ["country", {"COUNT": "*"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "impression"}],
    "group_by": ["country"],
    "order_by": [{"col": "COUNT(*)", "dir": "desc"}]
  },
  {
    "select": ["advertiser_id", {"SUM": "bid_price"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "impression"}],
    "group_by": ["advertiser_id"],
    "order_by": [{"col": "SUM(bid_price)", "dir": "desc"}]
  },
  {
    "select": ["publisher_id", {"COUNT": "*"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "serve"}],
    "group_by": ["publisher_id"],
    "order_by": [{"col": "COUNT(*)", "dir": "desc"}]
  },
  {
    "select": ["country", {"AVG": "total_price"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "purchase"}],
    "group_by": ["country"],
    "order_by": [{"col": "AVG(total_price)", "dir": "desc"}]
  },
  {
    "select": ["day", {"SUM": "bid_price"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "impression"}],
    "group_by": ["day"],
    "order_by": [{"col": "day", "dir": "asc"}]
  },
  {
    "select": ["day", {"COUNT": "*"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "serve"}],
    "group_by": ["day"],
    "order_by": [{"col": "day", "dir": "asc"}]
  },
  {
    "select": ["day", {"SUM": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "impression"},
      {"col": "country", "op": "eq", "val": "US"}
    ],
    "group_by": ["day"],
    "order_by": [{"col": "day", "dir": "asc"}]
  },
  {
    "select": ["day", {"AVG": "total_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "purchase"},
      {"col": "advertiser_id", "op": "eq", "val": "50"}
    ],
    "group_by": ["day"],
    "order_by": [{"col": "day", "dir": "asc"}]
  },
  {
    "select": ["day", {"COUNT": "*"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "serve"},
      {"col": "publisher_id", "op": "eq", "val": "100"}
    ],
    "group_by": ["day"],
    "order_by": [{"col": "day", "dir": "desc"}]
  },
  {
    "select": ["day", {"SUM": "total_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "purchase"},
      {"col": "country", "op": "eq", "val": "JP"}
    ],
    "group_by": ["day"],
    "order_by": [{"col": "day", "dir": "asc"}]
  },
  {
    "select": ["publisher_id", "country", {"COUNT": "*"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "serve"}],
    "group_by": ["publisher_id", "country"],
    "order_by": [{"col": "COUNT(*)", "dir": "desc"}]
  },
  {
    "select": ["publisher_id", {"SUM": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "impression"},
      {"col": "day", "op": "between", "val": ["2024-04-01", "2024-06-30"]}
    ],
    "group_by": ["publisher_id"],
    "order_by": [{"col": "SUM(bid_price)", "dir": "desc"}]
  },
  {
    "select": ["publisher_id", {"AVG": "total_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "purchase"},
      {"col": "country", "op": "eq", "val": "US"}
    ],
    "group_by": ["publisher_id"],
    "order_by": [{"col": "AVG(total_price)", "dir": "asc"}]
  },
  {
    "select": ["publisher_id", {"COUNT": "*"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "serve"},
      {"col": "advertiser_id", "op": "eq", "val": "25"}
    ],
    "group_by": ["publisher_id"],
    "order_by": [{"col": "COUNT(*)", "dir": "desc"}]
  },
  {
    "select": ["publisher_id", {"SUM": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "impression"},
      {"col": "day", "op": "eq", "val": "2024-08-15"}
    ],
    "group_by": ["publisher_id"],
    "order_by": [{"col": "SUM(bid_price)", "dir": "asc"}]
  },
  {
    "select": ["advertiser_id", "publisher_id", {"COUNT": "*"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "impression"}],
    "group_by": ["advertiser_id", "publisher_id"],
    "order_by": [{"col": "COUNT(*)", "dir": "desc"}]
  },
  {
    "select": ["advertiser_id", {"SUM": "total_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "purchase"},
      {"col": "day", "op": "between", "val": ["2024-02-01", "2024-04-30"]}
    ],
    "group_by": ["advertiser_id"],
    "order_by": [{"col": "SUM(total_price)", "dir": "desc"}]
  },
  {
    "select": ["advertiser_id", {"AVG": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "impression"},
      {"col": "country", "op": "eq", "val": "KR"}
    ],
    "group_by": ["advertiser_id"],
    "order_by": [{"col": "AVG(bid_price)", "dir": "asc"}]
  },
  {
    "select": ["advertiser_id", {"COUNT": "*"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "serve"},
      {"col": "publisher_id", "op": "eq", "val": "200"}
    ],
    "group_by": ["advertiser_id"],
    "order_by": [{"col": "COUNT(*)", "dir": "desc"}]
  },
  {
    "select": ["advertiser_id", {"SUM": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "impression"},
      {"col": "day", "op": "eq", "val": "2024-11-11"}
    ],
    "group_by": ["advertiser_id"],
    "order_by": [{"col": "SUM(bid_price)", "dir": "asc"}]
  },
  {
    "select": ["country", "type", {"COUNT": "*"}],
    "from": "events",
    "where": [{"col": "day", "op": "between", "val": ["2024-01-01", "2024-12-31"]}],
    "group_by": ["country", "type"],
    "order_by": [{"col": "COUNT(*)", "dir": "desc"}]
  },
  {
    "select": ["country", {"SUM": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "impression"},
      {"col": "day", "op": "between", "val": ["2024-06-01", "2024-08-31"]}
    ],
    "group_by": ["country"],
    "order_by": [{"col": "SUM(bid_price)", "dir": "desc"}]
  },
  {
    "select": ["country", {"AVG": "total_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "purchase"},
      {"col": "advertiser_id", "op": "between", "val": ["1", "100"]}
    ],
    "group_by": ["country"],
    "order_by": [{"col": "AVG(total_price)", "dir": "asc"}]
  },
  {
    "select": ["country", {"COUNT": "*"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "serve"},
      {"col": "publisher_id", "op": "between", "val": ["500", "1000"]}
    ],
    "group_by": ["country"],
    "order_by": [{"col": "COUNT(*)", "dir": "desc"}]
  },
  {
    "select": ["country", {"SUM": "total_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "purchase"},
      {"col": "day", "op": "between", "val": ["2024-09-01", "2024-11-30"]}
    ],
    "group_by": ["country"],
    "order_by": [{"col": "SUM(total_price)", "dir": "asc"}]
  },
  {
    "select": ["day", "country", "advertiser_id", "publisher_id", "type", {"COUNT": "*"}],
    "from": "events",
    "group_by": ["day", "country", "advertiser_id", "publisher_id", "type"]
  },
  {
    "select": ["country", "advertiser_id", {"SUM": "bid_price"}],
    "from": "events",
    "where": [{"col": "type", "op": "eq", "val": "impression"}],
    "group_by": ["country", "advertiser_id"],
    "order_by": [{"col": "SUM(bid_price)", "dir": "desc"}]
  },
  {
    "select": ["publisher_id", "type", {"AVG": "total_price"}],
    "from": "events",
    "where": [{"col": "day", "op": "between", "val": ["2024-01-01", "2024-12-31"]}],
    "group_by": ["publisher_id", "type"],
    "order_by": [{"col": "AVG(total_price)", "dir": "asc"}]
  },
  {
    "select": ["day", "type", {"COUNT": "*"}],
    "from": "events",
    "where": [{"col": "country", "op": "in", "val": ["US", "JP", "KR", "CA", "IN"]}],
    "group_by": ["day", "type"],
    "order_by": [{"col": "COUNT(*)", "dir": "desc"}]
  },
  {
    "select": ["advertiser_id", "country", "publisher_id", {"SUM": "bid_price"}],
    "from": "events",
    "where": [
      {"col": "type", "op": "eq", "val": "impression"},
      {"col": "day", "op": "between", "val": ["2024-06-01", "2024-08-31"]}
    ],
    "group_by": ["advertiser_id", "country", "publisher_id"],
    "order_by": [{"col": "SUM(bid_price)", "dir": "desc"}]
  }
]